In [ ]:
import gmsh
gmsh.initialize()
using LinearAlgebra, SparseArrays

In [ ]:
struct PlaneStressProblem
    name::String
    E::Float64
    ν::Float64
    b::Float64
end

In [ ]:
function displacementConstraint(groupOfLines; ux=1im, uy=1im)
    phg = gmsh.model.addPhysicalGroup(1, groupOfLines)
    gmsh.model.setPhysicalName(1, phg, "support")
    bc0 = phg, ux, uy
    return bc0
end

function traction(groupOfLines; fx=0, fy=0, thickness=1)
    phg = gmsh.model.addPhysicalGroup(1, groupOfLines)
    gmsh.model.setPhysicalName(1, phg, "load")
    ld0 = phg, fx, fy, thickness
    return ld0
end

In [ ]:
# Végeselemes felosztás elvégzése
function generateMesh(problem, surf, elemSize; approxOrder=1, algorithm=6, quadrangle=0, internalNodes=0)
    gmsh.model.setCurrent(problem.name)
    # lekérjük az összes csomópontot
    all = gmsh.model.getEntities(0)
    # megadjuk, hogy a csomóponthoz rendelt eleméret mekkora legyen
    gmsh.model.mesh.setSize(all, elemSize)
    # kiválasztjuk a 8-as számú hálózó algoritmust a 2D-s sf1 felülethez
    gmsh.model.mesh.setAlgorithm(2, surf, algorithm)
    # legeneráljuk a hálót a felület kontúrjához (1D-s)
    gmsh.model.mesh.generate(1)
    # legeneráljuk a hálót a felülethez (2D-s)
    gmsh.model.mesh.generate(2)
    # a legenerált háromszög elemeket négyszög elemekké alakítjuk
    if quadrangle
        gmsh.model.mesh.recombine()
    end
    # másodfokú elemekhez:
    # belső csomópontok használata
    if internalNodes
        gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 0) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
    else
        gmsh.option.setNumber("Mesh.SecondOrderIncomplete", 1) # 0:belső csomópontokkal 1:csak éleken lévő csomópontokkal
    end
    # közelítés fokszáma (1-től 5-ig)
    gmsh.model.mesh.setOrder(approxOrder)
end

In [ ]:
# Merevségi mátrix felépítése
function stiffnessMatrixPlaneStress(problem)
    # anyagállandók mátrixa
    E = problem.E
    ν = problem.ν
    b = problem.b
    D = E / (1 - ν^2) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2] # ÁSF feladat
    #D = E / ((1 + ν) * (1 - 2ν)) * [1-ν ν 0; ν 1-ν 0; 0 0 (1-2ν)/2] # SA feladat
    # modell kiválasztása
    gmsh.model.setCurrent(problem.name)
    # csomópontok sorszámának lekérése
    #nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
    # végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    # a lefoglalandó memória méretének becslése
    lengthOfIJV = sum([(div(length(elemNodeTags[i]), length(elemTags[i])) * 2)^2 * length(elemTags[i]) for i in 1:length(elemTags)])
    nn = []
    I = []
    J = []
    V = []
    V = convert(Vector{Float64}, V)
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)
    for i in 1:length(elemTypes)
        et = elemTypes[i]
        elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
        intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order))
        numIntPoints = length(intWeights)
        comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
        ∇h = reshape(dfun, :, numIntPoints)
        nnet = zeros(Int, length(elemTags[i]), numNodes)
        invJac = zeros(3, 3numIntPoints)
        Iidx = zeros(Int, numNodes * 2, numNodes * 2)
        Jidx = zeros(Int, numNodes * 2, numNodes * 2)
        for k in 1:numNodes*2, l in 1:numNodes*2
            Iidx[k, l] = l
            Jidx[k, l] = k
        end
        ∂h = zeros(3, numNodes * numIntPoints) # ∂h-t mindig csak felül kellene írni, nem kell újra és újra memóriát foglalni neki.
        B = zeros(3 * numIntPoints, 2 * numNodes) # B-t mindig csak felül kellene írni?
        K1 = zeros(2 * numNodes, 2 * numNodes)
        nn2 = zeros(Int, 2 * numNodes)
        for j in 1:length(elemTags[i])
            elem = elemTags[i][j]
            for k in 1:numNodes
                nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
            end
            jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
            Jac = reshape(jac, 3, :)
            for k in 1:numIntPoints
                invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
            end
            ∂h *= 0
            for k in 1:numIntPoints, l in 1:numNodes
                ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
            end
            B *= 0
            for k in 1:numIntPoints, l in 1:numNodes
                B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
                B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
            end
            K1 *= 0
            for k in 1:numIntPoints
                B1 = B[k*3-2:k*3, 1:2*numNodes]
                K1 += B1' * D * B1 * jacDet[k] * b * intWeights[k]
            end
            nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
            nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
            append!(I, nn2[Iidx[:]])
            append!(J, nn2[Jidx[:]])
            append!(V, K1[:])
        end
        push!(nn, nnet)
    end
    K = sparse(I, J, V)
    return K
end

In [ ]:
function applyBoundaryConditions!(problem, stiffMat, supports, tractions)
    gmsh.model.setCurrent(problem.name)
    dof, dof = size(stiffMat)
    fp = zeros(dof)
    for n in 1:length(tractions)
        phg, fx, fy, b = tractions[n]
        f = [fx, fy]
        tags = gmsh.model.getEntitiesForPhysicalGroup(1, phg)
        for i ∈ 1:length(tags)
            elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(1, tags[i])
            if length(elementTypes) != 1
                error("A peremen nem csak egyfajta végeselem van!")
            end
            elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementTypes[1])
            nnoe = reshape(elemNodeTags[1], numNodes, :)'
            intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(elementTypes[1], "Gauss" * string(order))
            numIntPoints = length(intWeights)
            comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "Lagrange")
            h = reshape(fun, :, numIntPoints)
            #comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[1], intPoints, "GradLagrange")
            #∇h = reshape(dfun, :, numIntPoints)
            H = zeros(2 * numIntPoints, 2 * numNodes)
            for j in 1:numIntPoints
                for k in 1:numNodes
                    H[j*2-1, k*2-1] = h[k, j]
                    H[j*2-0, k*2-0] = h[k, j]
                end
            end
            f1 = zeros(2 * numNodes)
            nn2 = zeros(Int, 2 * numNodes)
            for l in 1:length(elementTags[1])
                elem = elementTags[1][l]
                jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                Jac = reshape(jac, 3, :)
                f1 *= 0
                for j in 1:numIntPoints
                    H1 = H[j*2-1:j*2, 1:2*numNodes]
                    Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2)
                    f1 += H1' * f * Ja * b * intWeights[j]
                end
                nn2[1:2:2*numNodes] = 2 * nnoe[l, 1:numNodes] .- 1
                nn2[2:2:2*numNodes] = 2 * nnoe[l, 1:numNodes]
                fp[nn2] += f1
            end
        end

    end

    for i in 1:length(supports)
        phg, ux, uy = supports[i]
        nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, phg)
        if ux != 1im
            nodeTagsX = copy(nodeTags)
            nodeTagsX *= 2
            nodeTagsX .-= 1
            f0 = spzeros(dof, length(nodeTagsX))
            f0 = stiffMat[:, nodeTagsX] * ux
            f0 = sum(f0, dims=2)
            fp -= f0
        end
        if uy != 1im
            nodeTagsY = copy(nodeTags)
            nodeTagsY *= 2
            f0 = spzeros(dof, length(nodeTagsY))
            f0 = stiffMat[:, nodeTagsY] * uy
            f0 = sum(f0, dims=2)
            fp -= f0
        end
    end
    for i in 1:length(supports)
        phg, ux, uy = supports[i]
        nodeTags, coord = gmsh.model.mesh.getNodesForPhysicalGroup(1, phg)
        if ux != 1im
            nodeTagsX = copy(nodeTags)
            nodeTagsX *= 2
            nodeTagsX .-= 1
            for j ∈ nodeTagsX
                #display(j)
                stiffMat[j, :] .= 0
                stiffMat[:, j] .= 0
                stiffMat[j, j] = 1
                fp[j] = ux
            end
        end
        display("---------------------------------")
        if uy != 1im
            nodeTagsY = copy(nodeTags)
            nodeTagsY *= 2
            for j ∈ nodeTagsY
                #display(j)
                stiffMat[j, :] .= 0
                stiffMat[:, j] .= 0
                stiffMat[j, j] = 1
                fp[j] = uy
            end
        end
    end

    dropzeros!(stiffMat)
    return stiffMat, fp
end

In [ ]:
function solveDisplacement(K, f)
    return K \ f
end

In [ ]:
# Feszültségek számítása
function solveStressPlaneStress(problem, q)
    E = problem.E
    ν = problem.ν
    D = E / (1 - ν^2) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2] # ÁSF feladat
    gmsh.model.setCurrent(problem.name)
    # csomópontok sorszámának lekérése
    nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
    # végeselemek típusának, sorszámának és kapcsolati mátrixának (connectivity matrix) lekérése
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    numElem = Int[]
    σ = Vector{Float64}[]
    #σx = Vector{Float64}[]
    #σy = Vector{Float64}[]
    #σxy = Vector{Float64}[]
    for i in 1:length(elemTypes)
        et = elemTypes[i]
        elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
        s0 = zeros(3numNodes) # csak SA és ÁSF feladatnál, FSZ-nél már 4 kell
        nodeCoord = zeros(numNodes * 3)
        for k in 1:dim, j = 1:numNodes
            nodeCoord[k+(j-1)*3] = localNodeCoord[k+(j-1)*dim]
        end
        comp, dfun, ori = gmsh.model.mesh.getBasisFunctions(et, nodeCoord, "GradLagrange")
        ∇h = reshape(dfun, :, numNodes)
        nnet = zeros(Int, length(elemTags[i]), numNodes)
        invJac = zeros(3, 3numNodes)
        ∂h = zeros(3, numNodes * numNodes)
        B = zeros(3 * numNodes, 2 * numNodes)
        nn2 = zeros(Int, 2 * numNodes)
        for j in 1:length(elemTags[i])
            elem = elemTags[i][j]
            for k in 1:numNodes
                nnet[j, k] = elemNodeTags[i][(j-1)*numNodes+k]
            end
            jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, nodeCoord)
            Jac = reshape(jac, 3, :)
            for k in 1:numNodes
                invJac[1:3, 3*k-2:3*k] = inv(Jac[1:3, 3*k-2:3*k])'
            end
            ∂h *= 0
            for k in 1:numNodes, l in 1:numNodes
                ∂h[1:2, (k-1)*numNodes+l] = invJac[1:2, k*3-2:k*3-1] * ∇h[l*3-2:l*3-1, k] #??????????????????
            end
            B *= 0
            for k in 1:numNodes, l in 1:numNodes
                B[k*3-0, l*2-0] = B[k*3-2, l*2-1] = ∂h[1, (k-1)*numNodes+l]
                B[k*3-0, l*2-1] = B[k*3-1, l*2-0] = ∂h[2, (k-1)*numNodes+l]
            end
            push!(numElem, elem)
            nn2[1:2:2*numNodes] = 2 * nnet[j, 1:numNodes] .- 1
            nn2[2:2:2*numNodes] = 2 * nnet[j, 1:numNodes]
            s = zeros(9numNodes) # tenzornak 9 eleme van
            #sx = zeros(numNodes)
            #sy = zeros(numNodes)
            #sxy = zeros(numNodes)
            for k in 1:numNodes
                B1 = B[k*3-2:k*3, 1:2*numNodes]
                s0 = D * B1 * q[nn2]
                s[(k-1)*9+1:k*9] = [s0[1], s0[3], 0, s0[3], s0[2], 0, 0, 0, 0]
                #sx[k] = s0[1]
                #sy[k] = s0[2]
                #sxy[k] = s0[3]
            end
            push!(σ, s)
            #push!(σx, sx)
            #push!(σy, sy)
            #push!(σxy, sxy)
        end
    end
    return σ, numElem
end

In [ ]:
function showResultUvec(problem, q; name="uvec", visible=false)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elemTypes[1])
    nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
    non = length(nodeTags)
    uvec = gmsh.view.add(name)
    u = zeros(3 * non)
    for i in 1:length(nodeTags)
        u[3i-2] = q[2*nodeTags[i]-1]
        u[3i-1] = q[2*nodeTags[i]-0]
    end
    gmsh.view.addHomogeneousModelData(uvec, 0, problem.name, "NodeData", nodeTags, u, 0, 3)

    gmsh.view.option.setNumber(uvec, "AdaptVisualizationGrid", 1)
    gmsh.view.option.setNumber(uvec, "TargetError", -1e-4)
    gmsh.view.option.setNumber(uvec, "MaxRecursionLevel", order + 1)
    if visible == false
        gmsh.view.option.setNumber(uvec, "Visible", 0)
    end
    return uvec
end

In [ ]:
function showResultUX(problem, q; name="ux", visible=false)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elemTypes[1])
    nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
    non = length(nodeTags)
    ux = gmsh.view.add(name)
    u = zeros(non)
    for i in 1:length(nodeTags)
        u[i] = q[2*nodeTags[i]-1]
    end
    gmsh.view.addHomogeneousModelData(ux, 0, problem.name, "NodeData", nodeTags, u, 0, 1)

    gmsh.view.option.setNumber(ux, "AdaptVisualizationGrid", 1)
    gmsh.view.option.setNumber(ux, "TargetError", -1e-4)
    gmsh.view.option.setNumber(ux, "MaxRecursionLevel", order + 1)
    if visible == false
        gmsh.view.option.setNumber(ux, "Visible", 0)
    end
    return ux
end

In [ ]:
function showResultUY(problem, q; name="uy", visible=false)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elemTypes[1])
    nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
    non = length(nodeTags)
    uy = gmsh.view.add(name)
    u = zeros(non)
    for i in 1:length(nodeTags)
        u[i] = q[2*nodeTags[i]-0]
    end
    gmsh.view.addHomogeneousModelData(uy, 0, problem.name, "NodeData", nodeTags, u, 0, 1)

    gmsh.view.option.setNumber(uy, "AdaptVisualizationGrid", 1)
    gmsh.view.option.setNumber(uy, "TargetError", -1e-4)
    gmsh.view.option.setNumber(uy, "MaxRecursionLevel", order + 1)
    if visible == false
        gmsh.view.option.setNumber(uy, "Visible", 0)
    end
    return uy
end

In [ ]:
function showResultS(problem, S; name="σ", visible=true, smooth=true)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elemTypes[1])
    σ, numElem = S
    S = gmsh.view.add(name)
    gmsh.view.addModelData(S, 0, problem.name, "ElementNodeData", numElem, σ, 0, 9)

    if smooth == true
        gmsh.plugin.setNumber("Smooth", "View", -1)
        gmsh.plugin.run("Smooth")
    end

    gmsh.view.option.setNumber(S, "AdaptVisualizationGrid", 1)
    gmsh.view.option.setNumber(S, "TargetError", -1e-4)
    gmsh.view.option.setNumber(S, "MaxRecursionLevel", order + 1)
    if visible == false
        gmsh.view.option.setNumber(S, "Visible", 0)
    end
    return S
end

In [ ]:
function showResultSX(problem, S; name="σx", visible=false, smooth=true)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elemTypes[1])
    σ, numElem = S
    S = gmsh.view.add(name)
    σx = []
    sizehint!(σx, length(numElem))
    for i in 1:length(σ)
        sx = zeros(div(length(σ[i]), 9))
        for j in 1:(div(length(σ[i]), 9))
            sx[j] = σ[i][9j-8]
        end
        push!(σx, sx)
    end

    gmsh.view.addModelData(S, 0, problem.name, "ElementNodeData", numElem, σx, 0, 1)

    if smooth == true
        gmsh.plugin.setNumber("Smooth", "View", -1)
        gmsh.plugin.run("Smooth")
    end

    gmsh.view.option.setNumber(S, "AdaptVisualizationGrid", 1)
    gmsh.view.option.setNumber(S, "TargetError", -1e-4)
    gmsh.view.option.setNumber(S, "MaxRecursionLevel", order + 1)
    if visible == false
        gmsh.view.option.setNumber(S, "Visible", 0)
    end
    return S
end

In [ ]:
function showResultSY(problem, S; name="σy", visible=false, smooth=true)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elemTypes[1])
    σ, numElem = S
    S = gmsh.view.add(name)
    σy = []
    sizehint!(σy, length(numElem))
    for i in 1:length(σ)
        sy = zeros(div(length(σ[i]), 9))
        for j in 1:(div(length(σ[i]), 9))
            sy[j] = σ[i][9j-4]
        end
        push!(σy, sy)
    end

    gmsh.view.addModelData(S, 0, problem.name, "ElementNodeData", numElem, σy, 0, 1)

    if smooth == true
        gmsh.plugin.setNumber("Smooth", "View", -1)
        gmsh.plugin.run("Smooth")
    end

    gmsh.view.option.setNumber(S, "AdaptVisualizationGrid", 1)
    gmsh.view.option.setNumber(S, "TargetError", -1e-4)
    gmsh.view.option.setNumber(S, "MaxRecursionLevel", order + 1)
    if visible == false
        gmsh.view.option.setNumber(S, "Visible", 0)
    end
    return S
end

In [ ]:
function showResultSXY(problem, S; name="τxy", visible=false, smooth=true)
    gmsh.model.setCurrent(problem.name)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
    elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elemTypes[1])
    σ, numElem = S
    S = gmsh.view.add(name)
    σxy = []
    sizehint!(σxy, length(numElem))
    for i in 1:length(σ)
        sxy = zeros(div(length(σ[i]), 9))
        for j in 1:(div(length(σ[i]), 9))
            sxy[j] = σ[i][9j-7]
        end
        push!(σxy, sxy)
    end

    gmsh.view.addModelData(S, 0, problem.name, "ElementNodeData", numElem, σxy, 0, 1)

    if smooth == true
        gmsh.plugin.setNumber("Smooth", "View", -1)
        gmsh.plugin.run("Smooth")
    end

    gmsh.view.option.setNumber(S, "AdaptVisualizationGrid", 1)
    gmsh.view.option.setNumber(S, "TargetError", -1e-4)
    gmsh.view.option.setNumber(S, "MaxRecursionLevel", order + 1)
    if visible == false
        gmsh.view.option.setNumber(S, "Visible", 0)
    end
    return S
end

In [ ]:
function plotOnPath(problem, path, field, points; numOfStep=0, name="path", visible=false)
    gmsh.model.setCurrent(problem.name)
    dataType, tags, data, time, numComponents = gmsh.view.getModelData(field, numOfStep)
    bounds = gmsh.model.getParametrizationBounds(1, path)
    bound1 = bounds[1][1]
    bound2 = bounds[2][1]
    step0 = (bound2 - bound1) / (points - 1)
    cv = zeros(4)
    CoordValue = []
    pt0 = gmsh.model.getValue(1, path, [bound1])
    for i in 1:points
        pt1 = gmsh.model.getValue(1, path, [bound1 + (i - 1) * step0])
        cv[1:3] = pt1 - pt0
        val, dis = gmsh.view.probe(field, pt1[1], pt1[2], pt1[3])
        if dis == 0
            if numComponents == 1
                v = val[1]
            elseif numComponents == 3
                v = √(val[1]^2 + val[1]^2 + val[1]^2)
            elseif numComponents == 9
                v = √(0.5 * ((val[1] - val[5])^2 + (val[5] - val[9])^2 + (val[9] - val[1])^2 + 6 * (val[2]^2 + val[3]^2 + val[6]^2)))
            else
                error("Vagy skalás vagy vektor vagy tenzor...")
            end
        else
            v = 0
        end
        cv[4] = v
        append!(CoordValue, cv)
    end
    pathView = gmsh.view.add(name)
    gmsh.view.addListData(pathView, "SP", points, CoordValue)

    gmsh.view.option.setNumber(pathView, "Type", 2)
    gmsh.view.option.setNumber(pathView, "Axes", 1)

    if visible == false
        gmsh.view.option.setNumber(pathView, "Visible", 0)
    end
    return pathView
end

In [ ]:
# Adatok
E = 2e5
ν = 0.3
thickness = 1
height = 100
base = 100
elemSize = 1 #22

approxOrder = 1
internalNodes = true
quadElements = true

problem = PlaneStressProblem("rectangle", E, ν, thickness)

In [ ]:
# Geometria megrajzolása
gmsh.model.add(problem.name)

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
#p23 = gmsh.model.occ.addPoint(base, height / 2, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
#l23 = gmsh.model.occ.addLine(p23, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])

l5 = gmsh.model.occ.addCircle(base / 2, height / 2, 0, min(base, height) / 4)
cl2 = gmsh.model.occ.addCurveLoop([l5])

sf1 = gmsh.model.occ.addPlaneSurface([cl1, cl2])
#display(sf1)

#cp1 = gmsh.model.occ.addPoint(base / 2, height / 2, 0)
#cl1 = gmsh.model.occ.addLine(p1, p3)
#cl2 = gmsh.model.occ.addLine(p2, p4)

#sf2 = gmsh.model.occ.fragment([(2, sf1)], [(1, cl1)])
#gmsh.model.occ.removeAllDuplicates()

#p5 = gmsh.model.occ.addPoint(0, height / 2, 0)
#p6 = gmsh.model.occ.addPoint(base, height / 2, 0)
#l5 = gmsh.model.occ.addLine(p5, p6)

gmsh.model.occ.synchronize()
#sf2

In [ ]:
p5 = gmsh.model.occ.addPoint(0, height / 2, 0)
p6 = gmsh.model.occ.addPoint(base / 3, height / 2, 0)
p7 = gmsh.model.occ.addPoint(3base / 4, height / 2, 0)
p8 = gmsh.model.occ.addPoint(base, height / 2, 0)
l6 = gmsh.model.occ.addLine(p5, p6)
l7 = gmsh.model.occ.addLine(p7, p8)
sf2 = gmsh.model.occ.fragment([(2, sf1)], [(1, l6), (1, l7)])
gmsh.model.occ.synchronize()
sf2

In [ ]:
ee = 1
innerline = gmsh.model.getEntitiesInBoundingBox(base / 4 - ee, height / 2 - ee, 0 - ee, 3base / 4 + ee, height / 2 + ee, 0 + ee, 1)

In [ ]:
gmsh.model.occ.remove(innerline, true)
gmsh.model.occ.synchronize()

In [ ]:
gmsh.write("rectangle.brep")

In [ ]:
supp = gmsh.model.getEntitiesInBoundingBox(0 - ee, 0 - ee, 0 - ee, 0 + ee, height + ee, 0 + ee, 1)
load = gmsh.model.getEntitiesInBoundingBox(0 - ee, height - ee, 0 - ee, base + ee, height + ee, 0 + ee, 1)

In [ ]:
supp1 = displacementConstraint([supp[i][2] for i in 1:length(supp)], ux=0, uy=0)
load1 = traction([load[i][2] for i in 1:length(load)], fx=0, fy=1, thickness=thickness)

In [ ]:
generateMesh(problem, sf1, elemSize, approxOrder=approxOrder, algorithm=8, quadrangle=quadElements, internalNodes=internalNodes)

In [ ]:
gmsh.model.mesh.removeDuplicateNodes()
gmsh.model.mesh.renumberNodes()

In [ ]:
# 4.12 verziótól...
#oldTags, newTags = gmsh.model.mesh.computeRenumbering("RCMK") # "RCMK", "Hilbert", "Metis"
#gmsh.model.mesh.renumberNodes(oldTags, newTags)

In [ ]:
gmsh.write("rectangle.msh")

In [ ]:
gmsh.fltk.run()

In [ ]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [ ]:
K = stiffnessMatrixPlaneStress(problem)

In [ ]:
K, f = applyBoundaryConditions!(problem, K, [supp1], [load1])

In [ ]:
det(K)

In [ ]:
q = solveDisplacement(K, f)

In [ ]:
S = solveStressPlaneStress(problem, q)

In [ ]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [ ]:
#u = showResultUvec(problem, q, name="uvec", visible=false)
#ux = showResultUX(problem, q, name="ux", visible=false)
#uy = showResultUY(problem, q, name="uy", visible=false)
#s = showResultS(problem, S, name="σ", visible=false, smooth=false)
#sx = showResultSX(problem, S, name="σx", visible=false, smooth=false)
sy = showResultSY(problem, S, name="σy", visible=true, smooth=false)
#sxy = showResultSXY(problem, S, name="τxy", visible=false, smooth=false)

In [ ]:
pp1 = gmsh.model.occ.addPoint(0, height / 2, 0)
pp2 = gmsh.model.occ.addPoint(base / 4, height / 2, 0)
pp3 = gmsh.model.occ.addPoint(base / 8, height / 4, 0)

pl1 = gmsh.model.occ.addLine(pp1, pp2)
pl2 = gmsh.model.occ.addSpline([pp1, pp3, pp2])


gmsh.model.occ.synchronize()

In [ ]:
#plotOnPath(problem, pl1, s, 100, name="σred", visible=false);

In [ ]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [ ]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()